In [10]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, recall_score, roc_auc_score, precision_score, f1_score
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

In [11]:
df = pd.read_csv('healthcare-dataset-stroke-data.csv')
for c in df.columns: 
    if len(df[c].unique())<10:print(c, df[c].unique()) 

gender ['Male' 'Female' 'Other']
hypertension [0 1]
heart_disease [1 0]
ever_married ['Yes' 'No']
work_type ['Private' 'Self-employed' 'Govt_job' 'children' 'Never_worked']
Residence_type ['Urban' 'Rural']
smoking_status ['formerly smoked' 'never smoked' 'smokes' 'Unknown']
stroke [1 0]


In [12]:
df.gender = df.gender.replace({'Male':0, 'Female':1, 'Other': 2}).astype(np.uint16)
df.ever_married = df.ever_married.replace({'Yes':1, 'No':0}).astype(np.uint16)
df.work_type = df.work_type.replace({'Private':0, 'Self-employed':1, 'Govt_job':2, 'children':3, 'Never_worked':4})
df.Residence_type = df.Residence_type.replace({'Urban':0, 'Rural':1}).astype(np.uint16)
df.drop('smoking_status', axis = 1, inplace = True)

In [13]:
df.isnull().sum()

id                     0
gender                 0
age                    0
hypertension           0
heart_disease          0
ever_married           0
work_type              0
Residence_type         0
avg_glucose_level      0
bmi                  201
stroke                 0
dtype: int64

201 Missing BMI values
Fill with descision Tree

In [14]:
bmi_pipe = Pipeline(steps = [
                                ('scale', StandardScaler()),
                                ('dt',RandomForestRegressor())
                            ])

In [15]:
X = df[['gender','age','bmi']].copy()
mask = X.bmi.isna()
bmi_missing = X[mask]
X = X[~mask]
y = X.pop('bmi')

In [16]:
bmi_pipe.fit(X,y)
bmi_pred = bmi_pipe.predict(bmi_missing[['gender', 'age']])
bmi_pred = pd.Series(bmi_pred, index = bmi_missing.index)

In [17]:
df.bmi.fillna(bmi_pred, inplace=True)
df.isna().sum()

id                   0
gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
stroke               0
dtype: int64

In [18]:
df.to_csv('bmi_filled.csv', index=False)